# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
### <center> **Examples on storage solutions for Big Data (PostgreSQL)** </center>
---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

#### Create SparkSession 


In [32]:
from pyspark.sql import SparkSession

# Hint for homework: .config("spark.jars.packages", "groupId:artifactId:version") \

spark = SparkSession.builder \
    .appName("SparkSQL-Storage-Solutions-Postgres-Write") \
    .master("spark://078b2e28e517:7077") \
    .config("spark.jars", "/home/jovyan/notebooks/jars/postgresql-42.7.5.jar") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

#### Load Netflix Dataset

In [3]:
from team_name.spark_utils import SparkUtils

columns_info = [("show_id", "string"),("type", "string"),("title", "string"),("director", "string"),("country", "string"),("date_added", "string"),("release_year", "integer"),("rating", "string"), ("duration", "string"),("listed_in", "string")]

netflix_schema = SparkUtils.generate_schema(columns_info)

netflix_df = spark.read \
                .schema(netflix_schema) \
                .option("header", "true") \
                .csv("/home/jovyan/notebooks/data/netflix1.csv")

netflix_df.printSchema()

netflix_df.show(5, truncate=False)

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)

+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|show_id|type   |title                           |director       |country      |date_added|release_year|rating|duration|listed_in                                                    |
+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|s1     |Movie  |Dick Johnson Is Dead           

#### Write DF in Postgres

In [5]:
jdbc_url = "jdbc:postgresql://postgres-iteso:5432/postgres"

netflix_df.write \
    .format("jdbc") \
    .option("url", jdbc_url) \
    .option("dbtable", "netflix") \
    .option("user", "postgres") \
    .option("password", "Admin@1234") \
    .option("driver", "org.postgresql.Driver") \
    .save()

print("Done")

Done


In [6]:
# Stop the SparkContext
sc.stop()

#### Create a DF from PostgreSQL

In [12]:
spark = SparkSession.builder \
    .appName("SparkSQL-Storage-Solutions-Postgres-Read") \
    .master("spark://078b2e28e517:7077") \
    .config("spark.jars", "/home/jovyan/notebooks/jars/postgresql-42.7.5.jar") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext

##### Set Database connection properties 

In [13]:
# Database connection properties
jdbc_url = "jdbc:postgresql://postgres-iteso:5432/postgres"
db_properties = {
      "user": "postgres",      
      "password": "Admin@1234",
      "driver": "org.postgresql.Driver"
  }

df = spark.read \
    .jdbc(url=jdbc_url, table="netflix", properties=db_properties)

df.printSchema()
df.show(5, False)

root
 |-- show_id: string (nullable = true)
 |-- type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- director: string (nullable = true)
 |-- country: string (nullable = true)
 |-- date_added: string (nullable = true)
 |-- release_year: integer (nullable = true)
 |-- rating: string (nullable = true)
 |-- duration: string (nullable = true)
 |-- listed_in: string (nullable = true)

+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|show_id|type   |title                           |director       |country      |date_added|release_year|rating|duration|listed_in                                                    |
+-------+-------+--------------------------------+---------------+-------------+----------+------------+------+--------+-------------------------------------------------------------+
|s1     |Movie  |Dick Johnson Is Dead           

In [9]:
sc.stop()